In [1]:
import akshare as ak
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
from chinese_calendar import is_workday
import datetime
import dateutil
import time
import os
from io import StringIO
from tqdm import tqdm

In [2]:
# 增加log
import logging

#create a logger
logger = logging.getLogger('mylogger')
#set logger level
logger.setLevel(logging.WARNING)
#or you can set one of the following level
#logger.setLevel(logging.INFO)
#logger.setLevel(logging.DEBUG)

log_path = 'D:\Personal\Quant\data\Akshare\Stock'
handler = logging.FileHandler(os.path.join(log_path, 'mylog.log'))

# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)

#write a Warning message
logger.warning('This is a WARNING message')

In [3]:
stocks_summary_dir = 'D:\Personal\Quant\data\Akshare\Stock\stocks_summary'
stocks_infos_dir = 'D:\Personal\Quant\data\Akshare\Stock\stocks_infos'
stocks_history_dir = 'D:\Personal\Quant\data\Akshare\Stock\stocks_history'

In [5]:
# A股股票信息总览
stock_info_a_code_name_df = ak.stock_info_a_code_name()
stock_info_a_code_name_df.to_csv(os.path.join(stocks_infos_dir, 'a_stocks_name.csv'), index=False)
stock_info_sh_name_code_df = ak.stock_info_sh_name_code(symbol="主板A股")
stock_info_sh_name_code_df.to_csv(os.path.join(stocks_infos_dir, 'a_sh_stocks_name.csv'), index=False)
stock_info_sz_name_code_df = ak.stock_info_sz_name_code(symbol="A股列表")
stock_info_sz_name_code_df.to_csv(os.path.join(stocks_infos_dir, 'a_sz_stocks_name.csv'), index=False)
stock_info_bj_name_code_df = ak.stock_info_bj_name_code()
stock_info_bj_name_code_df.to_csv(os.path.join(stocks_infos_dir, 'a_bj_stocks_name.csv'), index=False)

stock_info_sz_delist_df = ak.stock_info_sz_delist(symbol="终止上市公司")
stock_info_sz_delist_df.to_csv(os.path.join(stocks_infos_dir, 'a_sz_delist_stocks_name.csv'), index=False)
stock_info_sh_delist_df = ak.stock_info_sh_delist()
stock_info_sh_delist_df.to_csv(os.path.join(stocks_infos_dir, 'a_sh_delist_stocks_name.csv'), index=False)
stock_info_sz_change_name_df = ak.stock_info_sz_change_name(symbol="全称变更")
stock_info_sz_change_name_df.to_csv(os.path.join(stocks_infos_dir, 'a_sz_change_stocks_name.csv'), index=False)

print("Finished")

Finished


In [4]:
# 获取最近交易日信息
def get_trade_day():
    today = datetime.datetime.now()
    now = datetime.datetime.now().strftime("%H:%M")
    if now < '17:00':
        today = today + datetime.timedelta(days=-1)
    today_str = is_open(today)
    return today_str

def is_trade_day(date):
    if is_workday(date):
        if datetime.datetime.isoweekday(date) < 6:
            # 排除周六周日 1-5：周一到周五
            return True
    return False

def is_open(today):
    today_str = today.strftime("%Y%m%d")
    while not is_trade_day(today):
        today = today + datetime.timedelta(days=-1)
        today_str = today.strftime("%Y%m%d")
    return today.strftime("%Y%m%d")

In [103]:
# 日增：上交所股票总览
def inc_sse_summary(sse_summary_file):   
    # 除上市公司数量和市盈率外，以亿为单位
    trade_day = get_trade_day()
    today_stock_sse_summary_df = ak.stock_sse_summary()    
    if today_stock_sse_summary_df.empty:
        logger.warning('No today stock sse summary!')
        return
    
    # 日增数据更新必须在每天17：00后！！
    now_h = datetime.datetime.now().strftime("%H")
    if int(now_h) < 17:
        logger.warning("Please update after 17:00!")
        return
    
    report_day = today_stock_sse_summary_df.loc[today_stock_sse_summary_df['项目']=='报告时间', '股票'].values[0]    
    if trade_day != report_day:
        logger.warning('Trade day is not equal with Report Day!')
        trade_day = report_day
    today_stock_sse_summary_df.loc[:, '日期'] = trade_day
    if(os.path.exists(sse_summary_file)):
        stock_sse_summary_df = pd.read_csv(sse_summary_file)
        prev_trade_days = stock_sse_summary_df['日期'].unique()
        if trade_day in prev_trade_days:
            logger.warning('Trade day summary infos already exist!')
            print('Finished')
            return
        else:
            logger.info(f'Append infos of trade day: {trade_day}')
            today_stock_sse_summary_df.to_csv(sse_summary_file, mode='a', index=False, header=False)
    else:
        today_stock_sse_summary_df.to_csv(sse_summary_file, index=False)
    print('Finished')

In [116]:
inc_sse_summary(os.path.join(stocks_summary_dir, 'sse_summary.csv'))

Finished


In [127]:
# 日增：深交所股票总览
def inc_szse_summary(szse_summary_file):
    trade_day = get_trade_day()
    today_stock_szse_summary_df = ak.stock_szse_summary(date=trade_day)
    if today_stock_szse_summary_df.empty:
        logger.warning('No today stock szse summary!')
        return
    
    # 日增数据更新必须在每天17：00后！！
    now_h = datetime.datetime.now().strftime("%H")
    if int(now_h) < 17:
        logger.warning("Please update after 17:00!")
        return

    today_stock_szse_summary_df.loc[:, '日期'] = trade_day
    if(os.path.exists(szse_summary_file)):
        stock_szse_summary_df = pd.read_csv(szse_summary_file)
        prev_trade_days = stock_szse_summary_df['日期'].unique()
        if trade_day in prev_trade_days:
            logger.warning('Trade day summary infos already exist!')
            print('Finished')
            return
        else:
            logger.info(f'Append infos of trade day: {trade_day}')
            today_stock_szse_summary_df.to_csv(szse_summary_file, mode='a', index=False, header=False)
    print('Finished')

In [128]:
get_szse_summary(os.path.join(stocks_summary_dir, 'szse_summary.csv'))

Process 2008-01-02 00:00:00...
Process 2008-01-03 00:00:00...
Process 2008-01-04 00:00:00...
Process 2008-01-07 00:00:00...
Process 2008-01-08 00:00:00...
Process 2008-01-09 00:00:00...
Process 2008-01-10 00:00:00...
Process 2008-01-11 00:00:00...
Process 2008-01-14 00:00:00...
Process 2008-01-15 00:00:00...
Process 2008-01-16 00:00:00...
Process 2008-01-17 00:00:00...
Process 2008-01-18 00:00:00...
Process 2008-01-21 00:00:00...
Process 2008-01-22 00:00:00...
Process 2008-01-23 00:00:00...
Process 2008-01-24 00:00:00...
Process 2008-01-25 00:00:00...
Process 2008-01-28 00:00:00...
Process 2008-01-29 00:00:00...
Process 2008-01-30 00:00:00...
Process 2008-01-31 00:00:00...
Process 2008-02-01 00:00:00...
Process 2008-02-04 00:00:00...
Process 2008-02-05 00:00:00...
Process 2008-02-13 00:00:00...
Process 2008-02-14 00:00:00...
Process 2008-02-15 00:00:00...
Process 2008-02-18 00:00:00...
Process 2008-02-19 00:00:00...
Process 2008-02-20 00:00:00...
Process 2008-02-21 00:00:00...
Process 

Process 2009-02-06 00:00:00...
Process 2009-02-09 00:00:00...
Process 2009-02-10 00:00:00...
Process 2009-02-11 00:00:00...
Process 2009-02-12 00:00:00...
Process 2009-02-13 00:00:00...
Process 2009-02-16 00:00:00...
Process 2009-02-17 00:00:00...
Process 2009-02-18 00:00:00...
Process 2009-02-19 00:00:00...
Process 2009-02-20 00:00:00...
Process 2009-02-23 00:00:00...
Process 2009-02-24 00:00:00...
Process 2009-02-25 00:00:00...
Process 2009-02-26 00:00:00...
Process 2009-02-27 00:00:00...
Process 2009-03-02 00:00:00...
Process 2009-03-03 00:00:00...
Process 2009-03-04 00:00:00...
Process 2009-03-05 00:00:00...
Process 2009-03-06 00:00:00...
Process 2009-03-09 00:00:00...
Process 2009-03-10 00:00:00...
Process 2009-03-11 00:00:00...
Process 2009-03-12 00:00:00...
Process 2009-03-13 00:00:00...
Process 2009-03-16 00:00:00...
Process 2009-03-17 00:00:00...
Process 2009-03-18 00:00:00...
Process 2009-03-19 00:00:00...
Process 2009-03-20 00:00:00...
Process 2009-03-23 00:00:00...
Process 

Process 2010-03-09 00:00:00...
Process 2010-03-10 00:00:00...
Process 2010-03-11 00:00:00...
Process 2010-03-12 00:00:00...
Process 2010-03-15 00:00:00...
Process 2010-03-16 00:00:00...
Process 2010-03-17 00:00:00...
Process 2010-03-18 00:00:00...
Process 2010-03-19 00:00:00...
Process 2010-03-22 00:00:00...
Process 2010-03-23 00:00:00...
Process 2010-03-24 00:00:00...
Process 2010-03-25 00:00:00...
Process 2010-03-26 00:00:00...
Process 2010-03-29 00:00:00...
Process 2010-03-30 00:00:00...
Process 2010-03-31 00:00:00...
Process 2010-04-01 00:00:00...
Process 2010-04-02 00:00:00...
Process 2010-04-06 00:00:00...
Process 2010-04-07 00:00:00...
Process 2010-04-08 00:00:00...
Process 2010-04-09 00:00:00...
Process 2010-04-12 00:00:00...
Process 2010-04-13 00:00:00...
Process 2010-04-14 00:00:00...
Process 2010-04-15 00:00:00...
Process 2010-04-16 00:00:00...
Process 2010-04-19 00:00:00...
Process 2010-04-20 00:00:00...
Process 2010-04-21 00:00:00...
Process 2010-04-22 00:00:00...
Process 

Process 2011-04-13 00:00:00...
Process 2011-04-14 00:00:00...
Process 2011-04-15 00:00:00...
Process 2011-04-18 00:00:00...
Process 2011-04-19 00:00:00...
Process 2011-04-20 00:00:00...
Process 2011-04-21 00:00:00...
Process 2011-04-22 00:00:00...
Process 2011-04-25 00:00:00...
Process 2011-04-26 00:00:00...
Process 2011-04-27 00:00:00...
Process 2011-04-28 00:00:00...
Process 2011-04-29 00:00:00...
Process 2011-05-03 00:00:00...
Process 2011-05-04 00:00:00...
Process 2011-05-05 00:00:00...
Process 2011-05-06 00:00:00...
Process 2011-05-09 00:00:00...
Process 2011-05-10 00:00:00...
Process 2011-05-11 00:00:00...
Process 2011-05-12 00:00:00...
Process 2011-05-13 00:00:00...
Process 2011-05-16 00:00:00...
Process 2011-05-17 00:00:00...
Process 2011-05-18 00:00:00...
Process 2011-05-19 00:00:00...
Process 2011-05-20 00:00:00...
Process 2011-05-23 00:00:00...
Process 2011-05-24 00:00:00...
Process 2011-05-25 00:00:00...
Process 2011-05-26 00:00:00...
Process 2011-05-27 00:00:00...
Process 

Process 2012-05-16 00:00:00...
Process 2012-05-17 00:00:00...
Process 2012-05-18 00:00:00...
Process 2012-05-21 00:00:00...
Process 2012-05-22 00:00:00...
Process 2012-05-23 00:00:00...
Process 2012-05-24 00:00:00...
Process 2012-05-25 00:00:00...
Process 2012-05-28 00:00:00...
Process 2012-05-29 00:00:00...
Process 2012-05-30 00:00:00...
Process 2012-05-31 00:00:00...
Process 2012-06-01 00:00:00...
Process 2012-06-04 00:00:00...
Process 2012-06-05 00:00:00...
Process 2012-06-06 00:00:00...
Process 2012-06-07 00:00:00...
Process 2012-06-08 00:00:00...
Process 2012-06-11 00:00:00...
Process 2012-06-12 00:00:00...
Process 2012-06-13 00:00:00...
Process 2012-06-14 00:00:00...
Process 2012-06-15 00:00:00...
Process 2012-06-18 00:00:00...
Process 2012-06-19 00:00:00...
Process 2012-06-20 00:00:00...
Process 2012-06-21 00:00:00...
Process 2012-06-25 00:00:00...
Process 2012-06-26 00:00:00...
Process 2012-06-27 00:00:00...
Process 2012-06-28 00:00:00...
Process 2012-06-29 00:00:00...
Process 

...

Process 2018-11-26 00:00:00...
Process 2018-11-27 00:00:00...
Process 2018-11-28 00:00:00...
Process 2018-11-29 00:00:00...
Process 2018-11-30 00:00:00...
Process 2018-12-03 00:00:00...
Process 2018-12-04 00:00:00...
Process 2018-12-05 00:00:00...
Process 2018-12-06 00:00:00...
Process 2018-12-07 00:00:00...
Process 2018-12-10 00:00:00...
Process 2018-12-11 00:00:00...
Process 2018-12-12 00:00:00...
Process 2018-12-13 00:00:00...
Process 2018-12-14 00:00:00...
Process 2018-12-17 00:00:00...
Process 2018-12-18 00:00:00...
Process 2018-12-19 00:00:00...
Process 2018-12-20 00:00:00...
Process 2018-12-21 00:00:00...
Process 2018-12-24 00:00:00...
Process 2018-12-25 00:00:00...
Process 2018-12-26 00:00:00...
Process 2018-12-27 00:00:00...
Process 2018-12-28 00:00:00...
Process 2019-01-02 00:00:00...
Process 2019-01-03 00:00:00...
Process 2019-01-04 00:00:00...
Process 2019-01-07 00:00:00...
Process 2019-01-08 00:00:00...
Process 2019-01-09 00:00:00...
Process 2019-01-10 00:00:00...
Process 

Process 2019-12-26 00:00:00...
Process 2019-12-27 00:00:00...
Process 2019-12-30 00:00:00...
Process 2019-12-31 00:00:00...
Process 2020-01-02 00:00:00...
Process 2020-01-03 00:00:00...
Process 2020-01-06 00:00:00...
Process 2020-01-07 00:00:00...
Process 2020-01-08 00:00:00...
Process 2020-01-09 00:00:00...
Process 2020-01-10 00:00:00...
Process 2020-01-13 00:00:00...
Process 2020-01-14 00:00:00...
Process 2020-01-15 00:00:00...
Process 2020-01-16 00:00:00...
Process 2020-01-17 00:00:00...
Process 2020-01-20 00:00:00...
Process 2020-01-21 00:00:00...
Process 2020-01-22 00:00:00...
Process 2020-01-23 00:00:00...
Process 2020-02-03 00:00:00...
Process 2020-02-04 00:00:00...
Process 2020-02-05 00:00:00...
Process 2020-02-06 00:00:00...
Process 2020-02-07 00:00:00...
Process 2020-02-10 00:00:00...
Process 2020-02-11 00:00:00...
Process 2020-02-12 00:00:00...
Process 2020-02-13 00:00:00...
Process 2020-02-14 00:00:00...
Process 2020-02-17 00:00:00...
Process 2020-02-18 00:00:00...
Process 

Process 2021-01-28 00:00:00...
Process 2021-01-29 00:00:00...
Process 2021-02-01 00:00:00...
Process 2021-02-02 00:00:00...
Process 2021-02-03 00:00:00...
Process 2021-02-04 00:00:00...
Process 2021-02-05 00:00:00...
Process 2021-02-08 00:00:00...
Process 2021-02-09 00:00:00...
Process 2021-02-10 00:00:00...
Process 2021-02-18 00:00:00...
Process 2021-02-19 00:00:00...
Process 2021-02-22 00:00:00...
Process 2021-02-23 00:00:00...
Process 2021-02-24 00:00:00...
Process 2021-02-25 00:00:00...
Process 2021-02-26 00:00:00...
Process 2021-03-01 00:00:00...
Process 2021-03-02 00:00:00...
Process 2021-03-03 00:00:00...
Process 2021-03-04 00:00:00...
Process 2021-03-05 00:00:00...
Process 2021-03-08 00:00:00...
Process 2021-03-09 00:00:00...
Process 2021-03-10 00:00:00...
Process 2021-03-11 00:00:00...
Process 2021-03-12 00:00:00...
Process 2021-03-15 00:00:00...
Process 2021-03-16 00:00:00...
Process 2021-03-17 00:00:00...
Process 2021-03-18 00:00:00...
Process 2021-03-19 00:00:00...
Process 

Process 2022-03-08 00:00:00...
Process 2022-03-09 00:00:00...
Process 2022-03-10 00:00:00...
Process 2022-03-11 00:00:00...
Process 2022-03-14 00:00:00...
Process 2022-03-15 00:00:00...
Process 2022-03-16 00:00:00...
Process 2022-03-17 00:00:00...
Process 2022-03-18 00:00:00...
Process 2022-03-21 00:00:00...
Process 2022-03-22 00:00:00...
Process 2022-03-23 00:00:00...
Process 2022-03-24 00:00:00...
Process 2022-03-25 00:00:00...
Process 2022-03-28 00:00:00...
Process 2022-03-29 00:00:00...
Process 2022-03-30 00:00:00...
Process 2022-03-31 00:00:00...
Process 2022-04-01 00:00:00...
Process 2022-04-06 00:00:00...
Process 2022-04-07 00:00:00...
Process 2022-04-08 00:00:00...
Process 2022-04-11 00:00:00...
Process 2022-04-12 00:00:00...
Process 2022-04-13 00:00:00...
Process 2022-04-14 00:00:00...
Process 2022-04-15 00:00:00...
Process 2022-04-18 00:00:00...
Process 2022-04-19 00:00:00...
Process 2022-04-20 00:00:00...
Process 2022-04-21 00:00:00...
Process 2022-04-22 00:00:00...
Process 

Process 2023-04-10 00:00:00...
Process 2023-04-11 00:00:00...
Process 2023-04-12 00:00:00...
Process 2023-04-13 00:00:00...
Process 2023-04-14 00:00:00...
Process 2023-04-17 00:00:00...
Process 2023-04-18 00:00:00...
Process 2023-04-19 00:00:00...
Process 2023-04-20 00:00:00...
Process 2023-04-21 00:00:00...
Process 2023-04-24 00:00:00...
Process 2023-04-25 00:00:00...
Process 2023-04-26 00:00:00...
Process 2023-04-27 00:00:00...
Process 2023-04-28 00:00:00...
Process 2023-05-04 00:00:00...
Process 2023-05-05 00:00:00...
Process 2023-05-08 00:00:00...
Process 2023-05-09 00:00:00...
Process 2023-05-10 00:00:00...
Process 2023-05-11 00:00:00...
Process 2023-05-12 00:00:00...
Process 2023-05-15 00:00:00...
Process 2023-05-16 00:00:00...
Process 2023-05-17 00:00:00...
Process 2023-05-18 00:00:00...
Process 2023-05-19 00:00:00...
Process 2023-05-22 00:00:00...
Process 2023-05-23 00:00:00...
Process 2023-05-24 00:00:00...
Process 2023-05-25 00:00:00...
Process 2023-05-26 00:00:00...
Process 

In [ ]:
# 日增：深交所股票总览
def inc_szse_summary(szse_summary_file):
    trade_day = get_trade_day()
    today_stock_szse_summary_df = ak.stock_szse_summary(date=trade_day)
    if today_stock_szse_summary_df.empty:
        logger.warning('No today stock szse summary!')
        return
    today_stock_szse_summary_df.loc[:, '日期'] = trade_day
    if(os.path.exists(szse_summary_file)):
        stock_szse_summary_df = pd.read_csv(szse_summary_file)
        prev_trade_days = stock_szse_summary_df['日期'].unique()
        if trade_day in prev_trade_days:
            logger.warning('Trade day summary infos already exist!')
            print('Finished')
            return
        else:
            logger.info(f'Append infos of trade day: {trade_day}')
            today_stock_szse_summary_df.to_csv(szse_summary_file, mode='a', index=False, header=False)
    print('Finished')

In [ ]:
inc_szse_summary(os.path.join(stocks_summary_dir, 'szse_summary.csv'))

In [27]:
# 汇总/月增：每月统计年鉴
# NOTE：API可能发生了变更，2018年之前的历史数据没有拿到
def get_szse_sector_summary_current_month(szse_sector_summary_current_month_file, begin_day='20180401'):
    begin_day = datetime.datetime.strptime(begin_day, '%Y%m%d')
    end_day = get_trade_day()
    end_day = datetime.datetime.strptime(end_day, '%Y%m%d')
    record_day = begin_day
    
    szse_sector_summary = pd.DataFrame()
    i = 1
    while record_day < end_day:
        print(f'Process {record_day}...')
        day_str = datetime.datetime.strftime(record_day, '%Y%m%d')
        try:            
            stock_szse_summary_df = ak.stock_szse_sector_summary(symbol="当月", date=day_str[:-2])
            print(stock_szse_sector_summary_df)
            if not stock_szse_sector_summary_df.empty:
                stock_szse_summary_df.loc[:, '日期'] = day_str[:-2]
                szse_sector_summary = pd.concat([szse_sector_summary, stock_szse_summary_df], axis=0).reset_index(drop=True)                
        except:
            pass
        record_day += dateutil.relativedelta.relativedelta(months=i)        
        szse_sector_summary.to_csv(szse_sector_summary_current_month_file, mode='a', index=False)
    print("Finished")

In [28]:
get_szse_sector_summary_current_month(os.path.join(stocks_summary_dir, 'szse_sector_summary_current_month.csv'), begin_day='20220601')

Process 2018-04-01 00:00:00...
    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13 

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

...

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

    项目名称                   项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计  \
0     合计                     Total    19  9103954078642    100.00   
1   农林牧渔               Agriculture    19   195073102951      2.14   
2    采矿业                    Mining    19   161690702423      1.78   
3    制造业             Manufacturing    19  5826712679774     64.00   
4   水电煤气                 Utilities    19   106693613542      1.17   
5    建筑业              Construction    19   216126531155      2.37   
6   批发零售        Wholesale & Retail    19   242628892221      2.67   
7   运输仓储            Transportation    19   140061078206      1.54   
8   住宿餐饮         Hotels & Catering    19    14789626661      0.16   
9   信息技术                        IT    19   691533724364      7.60   
10   金融业                   Finance    19   368534045943      4.05   
11   房地产               Real Estate    19   493545476445      5.42   
12  商务服务          Business Support    19   156220688882      1.72   
13  科研服务    Research & Development

In [50]:
stock_szse_sector_summary_df = ak.stock_szse_sector_summary(symbol="当月", date="202206")
print(stock_szse_sector_summary_df)

HTTPError: HTTP Error 403: Forbidden

In [47]:
# 个股信息
stock_individual_info_em_df = ak.stock_individual_info_em(symbol="000001")
print(stock_individual_info_em_df)
stock_individual_info_em_df = ak.stock_individual_info_em(symbol="000858")
print(stock_individual_info_em_df)

   item                value
0   总市值  218122520545.519989
1  流通市值       218118347718.0
2    行业                   银行
3  上市时间             19910403
4  股票代码               000001
5  股票简称                 平安银行
6   总股本        19405918198.0
7   流通股        19405546950.0
   item                value
0   总市值       637748195221.5
1  流通市值  637734706520.100098
2    行业                 酿酒行业
3  上市时间             19980427
4  股票代码               000858
5  股票简称                五 粮 液
6   总股本         3881608005.0
7   流通股         3881525907.0


In [48]:
# 行情报价，level-1数据
stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="000001")
print(stock_bid_ask_em_df)

          item       value
0       sell_5       11.28
1   sell_5_vol   219700.00
2       sell_4       11.27
3   sell_4_vol   274000.00
4       sell_3       11.26
5   sell_3_vol   361600.00
6       sell_2       11.25
7   sell_2_vol   190200.00
8       sell_1       11.24
9   sell_1_vol   273900.00
10       buy_1       11.23
11   buy_1_vol   182300.00
12       buy_2       11.22
13   buy_2_vol   687700.00
14       buy_3       11.21
15   buy_3_vol   989000.00
16       buy_4       11.20
17   buy_4_vol  1909300.00
18       buy_5       11.19
19   buy_5_vol   399200.00


In [49]:
# 实时行情数据，可能用于实盘
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
print(stock_zh_a_spot_em_df)

        序号      代码    名称    最新价    涨跌幅    涨跌额       成交量           成交额     振幅  \
0        1  300817  双飞股份  18.04  20.03   3.01  213557.0  3.597364e+08  24.02   
1        2  301192  泰祥股份  30.18  20.00   5.03  137979.0  3.791766e+08  25.05   
2        3  300657  弘信电子  22.69  19.99   3.78  608959.0  1.323636e+09  19.99   
3        4  301488   C豪恩  94.68  18.93  15.07  146024.0  1.266356e+09  21.79   
4        5  301141  中科磁业  63.00  18.24   9.72  142680.0  8.145994e+08  19.91   
...    ...     ...   ...    ...    ...    ...       ...           ...    ...   
5451  5452  603888   新华网  31.77  -9.95  -3.51  361623.0  1.175309e+09   8.08   
5452  5453  002791  坚朗五金  54.56 -10.00  -6.06   79582.0  4.480942e+08   9.98   
5453  5454  002553  南方精工  16.16 -10.02  -1.80   64100.0  1.035856e+08   0.00   
5454  5455  301006  迈拓股份  22.49 -12.18  -3.12   52864.0  1.239824e+08  13.67   
5455  5456  300389   艾比森  18.11 -13.56  -2.84  268438.0  4.711331e+08  13.94   

         最高  ...    量比    换手率  市盈率-动态  

In [8]:
# 汇总：历史行情数据-东财
def get_history_daily(stocks_name_file, history_daily_dir, adjust=''):
    stocks_name = pd.read_csv(stocks_name_file)['code'].tolist()
    stocks_name = [str(x).zfill(6) for x in stocks_name]
    
    start_date = '20000101'
    end_date = '20230706'
    
    for stock in stocks_name:
        print(f"Process {stock}...")
        stock_file = os.path.join(history_daily_dir, stock.zfill(6) + '.csv')        
        stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=stock, period="daily", start_date=start_date, end_date=end_date, adjust=adjust)
        stock_zh_a_hist_df['日期'] = stock_zh_a_hist_df['日期'].apply(lambda x: str(x).replace('-', ''))
        stock_zh_a_hist_df.to_csv(os.path.join(stock_file), index=False)
    
    print("Finished!")

In [9]:
daily_no_adjust_dir = os.path.join(stocks_history_dir, 'daily_no_adjust')
get_history_daily(os.path.join(stocks_infos_dir, 'a_stocks_name.csv'), daily_no_adjust_dir)
daily_post_adjust_dir = os.path.join(stocks_history_dir, 'daily_post_adjust')
get_history_daily(os.path.join(stocks_infos_dir, 'a_stocks_name.csv'), daily_post_adjust_dir, adjust='hfq')

Process 000001...
Process 000002...
Process 000004...
Process 000005...
Process 000006...
Process 000007...
Process 000008...
Process 000009...
Process 000010...
Process 000011...
Process 000012...
Process 000014...
Process 000016...
Process 000017...
Process 000019...
Process 000020...
Process 000021...
Process 000023...
Process 000025...
Process 000026...
Process 000027...
Process 000028...
Process 000029...
Process 000030...
Process 000031...
Process 000032...
Process 000034...
Process 000035...
Process 000036...
Process 000037...
Process 000038...
Process 000039...
Process 000040...
Process 000042...
Process 000045...
Process 000046...
Process 000048...
Process 000049...
Process 000050...
Process 000055...
Process 000056...
Process 000058...
Process 000059...
Process 000060...
Process 000061...
Process 000062...
Process 000063...
Process 000065...
Process 000066...
Process 000068...
Process 000069...
Process 000070...
Process 000078...
Process 000088...
Process 000089...
Process 00

Process 001222...
Process 001223...
Process 001225...
Process 001226...
Process 001227...
Process 001228...
Process 001229...
Process 001230...
Process 001231...
Process 001234...
Process 001236...
Process 001238...
Process 001255...
Process 001256...
Process 001258...
Process 001259...
Process 001260...
Process 001266...
Process 001267...
Process 001268...
Process 001269...
Process 001270...
Process 001278...
Process 001282...
Process 001283...
Process 001286...
Process 001287...
Process 001288...
Process 001289...
Process 001296...
Process 001298...
Process 001299...
Process 001300...
Process 001301...
Process 001308...
Process 001309...
Process 001311...
Process 001313...
Process 001314...
Process 001316...
Process 001317...
Process 001318...
Process 001319...
Process 001322...
Process 001323...
Process 001324...
Process 001328...
Process 001330...
Process 001331...
Process 001332...
Process 001333...
Process 001336...
Process 001337...
Process 001338...
Process 001339...
Process 00

Process 002400...
Process 002401...
Process 002402...
Process 002403...
Process 002404...
Process 002405...
Process 002406...
Process 002407...
Process 002408...
Process 002409...
Process 002410...
Process 002411...
Process 002412...
Process 002413...
Process 002414...
Process 002415...
Process 002416...
Process 002417...
Process 002418...
Process 002419...
Process 002420...
Process 002421...
Process 002422...
Process 002423...
Process 002424...
Process 002425...
Process 002426...
Process 002427...
Process 002428...
Process 002429...
Process 002430...
Process 002431...
Process 002432...
Process 002433...
Process 002434...
Process 002435...
Process 002436...
Process 002437...
Process 002438...
Process 002439...
Process 002440...
Process 002441...
Process 002442...
Process 002443...
Process 002444...
Process 002445...
Process 002446...
Process 002448...
Process 002449...
Process 002451...
Process 002452...
Process 002453...
Process 002454...
Process 002455...
Process 002456...
Process 00

Process 002886...
Process 002887...
Process 002888...
Process 002889...
Process 002890...
Process 002891...
Process 002892...
Process 002893...
Process 002895...
Process 002896...
Process 002897...
Process 002898...
Process 002899...
Process 002900...
Process 002901...
Process 002902...
Process 002903...
Process 002905...
Process 002906...
Process 002907...
Process 002908...
Process 002909...
Process 002910...
Process 002911...
Process 002912...
Process 002913...
Process 002915...
Process 002916...
Process 002917...
Process 002918...
Process 002919...
Process 002920...
Process 002921...
Process 002922...
Process 002923...
Process 002925...
Process 002926...
Process 002927...
Process 002928...
Process 002929...
Process 002930...
Process 002931...
Process 002932...
Process 002933...
Process 002935...
Process 002936...
Process 002937...
Process 002938...
Process 002939...
Process 002940...
Process 002941...
Process 002942...
Process 002943...
Process 002945...
Process 002946...
Process 00

Process 300326...
Process 300327...
Process 300328...
Process 300329...
Process 300331...
Process 300332...
Process 300333...
Process 300334...
Process 300335...
Process 300336...
Process 300337...
Process 300338...
Process 300339...
Process 300340...
Process 300341...
Process 300342...
Process 300343...
Process 300344...
Process 300345...
Process 300346...
Process 300347...
Process 300348...
Process 300349...
Process 300350...
Process 300351...
Process 300352...
Process 300353...
Process 300354...
Process 300355...
Process 300356...
Process 300357...
Process 300358...
Process 300359...
Process 300360...
Process 300363...
Process 300364...
Process 300365...
Process 300366...
Process 300368...
Process 300369...
Process 300370...
Process 300371...
Process 300373...
Process 300374...
Process 300375...
Process 300376...
Process 300377...
Process 300378...
Process 300379...
Process 300380...
Process 300381...
Process 300382...
Process 300383...
Process 300384...
Process 300385...
Process 30

...

Process 600519...
Process 600520...
Process 600521...
Process 600522...
Process 600523...
Process 600525...
Process 600526...
Process 600527...
Process 600528...
Process 600529...
Process 600530...
Process 600531...
Process 600533...
Process 600535...
Process 600536...
Process 600537...
Process 600538...
Process 600539...
Process 600540...
Process 600543...
Process 600545...
Process 600546...
Process 600547...
Process 600548...
Process 600549...
Process 600550...
Process 600551...
Process 600552...
Process 600556...
Process 600557...
Process 600558...
Process 600559...
Process 600560...
Process 600561...
Process 600562...
Process 600563...
Process 600565...
Process 600566...
Process 600567...
Process 600568...
Process 600569...
Process 600570...
Process 600571...
Process 600572...
Process 600573...
Process 600575...
Process 600576...
Process 600577...
Process 600578...
Process 600579...
Process 600580...
Process 600581...
Process 600582...
Process 600583...
Process 600584...
Process 60

Process 601216...
Process 601218...
Process 601222...
Process 601225...
Process 601226...
Process 601228...
Process 601229...
Process 601231...
Process 601233...
Process 601236...
Process 601238...
Process 601258...
Process 601279...
Process 601288...
Process 601298...
Process 601311...
Process 601318...
Process 601319...
Process 601326...
Process 601328...
Process 601330...
Process 601333...
Process 601336...
Process 601339...
Process 601360...
Process 601366...
Process 601368...
Process 601369...
Process 601375...
Process 601377...
Process 601388...
Process 601390...
Process 601398...
Process 601399...
Process 601456...
Process 601500...
Process 601512...
Process 601515...
Process 601518...
Process 601519...
Process 601528...
Process 601555...
Process 601566...
Process 601567...
Process 601568...
Process 601577...
Process 601579...
Process 601588...
Process 601595...
Process 601598...
Process 601599...
Process 601600...
Process 601601...
Process 601606...
Process 601607...
Process 60

Process 603527...
Process 603528...
Process 603529...
Process 603530...
Process 603533...
Process 603535...
Process 603536...
Process 603538...
Process 603551...
Process 603555...
Process 603556...
Process 603557...
Process 603558...
Process 603559...
Process 603565...
Process 603566...
Process 603567...
Process 603568...
Process 603569...
Process 603577...
Process 603578...
Process 603579...
Process 603580...
Process 603583...
Process 603585...
Process 603586...
Process 603587...
Process 603588...
Process 603589...
Process 603590...
Process 603595...
Process 603596...
Process 603598...
Process 603599...
Process 603600...
Process 603601...
Process 603602...
Process 603603...
Process 603605...
Process 603606...
Process 603607...
Process 603608...
Process 603609...
Process 603610...
Process 603611...
Process 603612...
Process 603613...
Process 603615...
Process 603616...
Process 603617...
Process 603618...
Process 603619...
Process 603626...
Process 603628...
Process 603629...
Process 60

Process 688099...
Process 688100...
Process 688101...
Process 688102...
Process 688103...
Process 688105...
Process 688106...
Process 688107...
Process 688108...
Process 688109...
Process 688110...
Process 688111...
Process 688112...
Process 688113...
Process 688114...
Process 688115...
Process 688116...
Process 688117...
Process 688118...
Process 688119...
Process 688120...
Process 688121...
Process 688122...
Process 688123...
Process 688125...
Process 688126...
Process 688127...
Process 688128...
Process 688129...
Process 688130...
Process 688131...
Process 688132...
Process 688133...
Process 688135...
Process 688136...
Process 688137...
Process 688138...
Process 688139...
Process 688141...
Process 688143...
Process 688146...
Process 688147...
Process 688148...
Process 688150...
Process 688151...
Process 688152...
Process 688153...
Process 688155...
Process 688156...
Process 688157...
Process 688158...
Process 688159...
Process 688160...
Process 688161...
Process 688162...
Process 68

Process 430047...
Process 430090...
Process 430139...
Process 430198...
Process 430300...
Process 430418...
Process 430425...
Process 430476...
Process 430478...
Process 430489...
Process 430510...
Process 430556...
Process 430564...
Process 430685...
Process 430718...
Process 830779...
Process 830799...
Process 830809...
Process 830832...
Process 830839...
Process 830879...
Process 830896...
Process 830946...
Process 830964...
Process 830974...
Process 831010...
Process 831039...
Process 831087...
Process 831152...
Process 831167...
Process 831195...
Process 831278...
Process 831304...
Process 831305...
Process 831370...
Process 831445...
Process 831526...
Process 831641...
Process 831689...
Process 831726...
Process 831768...
Process 831832...
Process 831834...
Process 831855...
Process 831856...
Process 831906...
Process 831961...
Process 832000...
Process 832023...
Process 832089...
Process 832110...
Process 832145...
Process 832149...
Process 832171...
Process 832175...
Process 83

In [ ]:
# 日增：历史行情数据-东财
def get_history_daily(stocks_name_file, history_daily_dir, start_date, end_date, adjust=''):
    stocks_name = pd.read_csv(stocks_name_file)['code'].tolist()
    stocks_name = [str(x).zfill(6) for x in stocks_name]
    
    for stock in tqdm(stocks_name):
        stock_file = os.path.join(history_daily_dir, stock.zfill(6) + '.csv')           
#         daily_stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=stock, period="daily", start_date=start_date, end_date=end_date, adjust=adjust)
        stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=stock, period="daily", start_date=start_date, end_date=end_date, adjust=adjust)
        print(stock)
        print(daily_stock_zh_a_hist_df)
        daily_stock_zh_a_hist_df['日期'] = daily_stock_zh_a_hist_df['日期'].apply(lambda x: str(x).replace('-', ''))
        stock_zh_a_hist_df = pd.read_csv(os.path.join(stock_file))
        stock_dates = stock_zh_a_hist_df['日期'].tolist()
        daily_stock_zh_a_hist_df = daily_stock_zh_a_hist_df.filter(lambda x: x['日期'] in stock_dates)
#         daily_stock_zh_a_hist_df.to_csv(os.path.join(stock_file), mode='a', index=False)
        print(daily_stock_zh_a_hist_df)
        assert 0
    
    print("Finished!")

In [ ]:
daily_no_adjust_dir = os.path.join(stocks_history_dir, 'daily_no_adjust')
get_history_daily(os.path.join(stocks_infos_dir, 'a_stocks_name.csv'), '20000101', '20230708', daily_no_adjust_dir)

In [5]:
def get_history_minute(stocks_name_file, history_minute_dir, minute=1, adjust=''):    
    if '_sh_' in stocks_name_file:
        prefix = 'sh'
        stocks_name = pd.read_csv(stocks_name_file)['证券代码'].tolist()
    elif '_sz_' in stocks_name_file:
        prefix = 'sz'
        stocks_name = pd.read_csv(stocks_name_file)['A股代码'].tolist()
    elif '_bj_' in stocks_name_file:
        stocks_name = pd.read_csv(stocks_name_file)['证券代码'].tolist()
        prefix = 'bj'
    else:
        prefix = ''
        logger.warning("stocks_name_file not found!")
        return
    
    stocks_name = [str(x).zfill(6) for x in stocks_name]
    for stock in tqdm(stocks_name):
        stock_zh_a_minute_df = ak.stock_zh_a_minute(symbol=prefix+stock, period=f'{minute}', adjust=adjust)
        stock_file = os.path.join(history_minute_dir, stock + '.csv')      
        if stock_zh_a_minute_df is not None:
            stock_zh_a_minute_df['day'] = stock_zh_a_minute_df['day'].apply(lambda x: str(x).replace('-', ''))
            stock_zh_a_minute_df.to_csv(stock_file, index=False)
        else:
            logger.warning(f'get_history_minute {prefix+stock} failed!')
    print(f"Finised {stocks_name_file} on {minute}!")

In [19]:
# 日增：腾讯财经3s tick数据，每天17：00后更新
def _inc_tick_3s_tx(stocks_name_file, history_tick_3s_tx_dir):
    today = datetime.datetime.now()
    now_h = datetime.datetime.now().strftime("%H")
    if int(now_h) < 17:
        print("Please update after 17:00!")
        return
    
    today = datetime.datetime.now().strftime("%Y%m%d")
    trade_day = get_trade_day()
#     if trade_day != today:
#         print("Only update today's 3s tick infos!")
#         return
    
    if '_sh_' in stocks_name_file:
        prefix = 'sh'
        stocks_name = pd.read_csv(stocks_name_file)['证券代码'].tolist()
    elif '_sz_' in stocks_name_file:
        prefix = 'sz'
        stocks_name = pd.read_csv(stocks_name_file)['A股代码'].tolist()
    elif '_bj_' in stocks_name_file:
        stocks_name = pd.read_csv(stocks_name_file)['证券代码'].tolist()
        prefix = 'bj'
    else:
        prefix = ''
        logger.warning("stocks_name_file not found!")
        return
    
    stocks_name = [str(x).zfill(6) for x in stocks_name]
    
    for stock in tqdm(stocks_name):
        stock_zh_a_tick_tx_js_df = ak.stock_zh_a_tick_tx_js(symbol=prefix+stock)
        if not stock_zh_a_tick_tx_js_df.empty:
            stock_zh_a_tick_tx_js_df.loc[:, '日期'] = trade_day
            stock_file = os.path.join(history_tick_3s_tx_dir, stock + '.csv')  
            stock_zh_a_tick_tx_js_df.to_csv(stock_file, index=False)
        else:
            logger.warning(f'get_history_minute {prefix+stock} failed!')


def inc_tick_3s_tx():
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        stocks_name_files = [os.path.join(stocks_infos_dir, f'a_{market}_stocks_name.csv') for market in ['sh', 'sz', 'bj']]
        history_tick_3s_tx_dir = os.path.join(stocks_history_dir, 'tick_no_adjust')
        trade_day = get_trade_day()
        history_tick_3s_tx_dir = os.path.join(history_tick_3s_tx_dir, trade_day)
#         os.makedirs(history_tick_3s_tx_dir)
        for stocks_name_file in stocks_name_files[1:]:
            _inc_tick_3s_tx(stocks_name_file, history_tick_3s_tx_dir)

    print("Finished")

In [20]:
inc_tick_3s_tx()

Please update after 17:00!
Please update after 17:00!
Finished


In [10]:
# history_minute_no_adjust_dir_dict = {m: os.path.join(stocks_history_dir, f'minute_{m}_no_adjust') for m in [1, 5, 15, 30, 60]}
# stocks_name_files = [os.path.join(stocks_infos_dir, f'a_{market}_stocks_name.csv') for market in ['sh', 'sz', 'bj']]
# for adjust in ['', 'hfq']:
#     for m, history_minute_dir in history_minute_no_adjust_dir_dict.items():
#         for stocks_name_file in stocks_name_files:
#             get_history_minute(stocks_name_file, history_minute_dir, minute=m, adjust=adjust)
# print("Finished")

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    stocks_name_files = [os.path.join(stocks_infos_dir, f'a_{market}_stocks_name.csv') for market in ['sh', 'sz', 'bj']]
    history_tick_3s_tx_dir = os.path.join(stocks_history_dir, 'tick_no_adjust')
    for stocks_name_file in stocks_name_files[:1]:
        inc_tick_3s_tx(stocks_name_file, history_tick_3s_tx_dir)

print("Finished")

  6%|████▍                                                                         | 95/1679 [12:09<3:22:39,  7.68s/it]


OSError: Cannot save file into a non-existent directory: 'D:\Personal\Quant\data\Akshare\Stock\stocks_history\tick_no_adjust\20230714'

In [11]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="301202", period="daily", start_date="20170301", end_date='20230711', adjust="")
print(stock_zh_a_hist_df)

           日期     开盘     收盘     最高     最低     成交量           成交额     振幅    涨跌幅  \
0  2023-07-05  45.00  42.93  50.00  42.90  239469  1.090900e+09  27.50  66.27   
1  2023-07-06  39.31  44.12  45.60  38.97  198342  8.235890e+08  15.44   2.77   
2  2023-07-07  42.50  38.62  43.22  38.58  153829  6.333845e+08  10.52 -12.47   
3  2023-07-10  36.83  39.73  40.58  35.63  146024  5.539852e+08  12.82   2.87   
4  2023-07-11  40.50  40.68  44.27  39.80  191333  7.998159e+08  11.25   2.39   

     涨跌额    换手率  
0  17.11  74.12  
1   1.19  61.39  
2  -5.50  47.61  
3   1.11  45.19  
4   0.95  59.22  


In [ ]:
import time

def test_real_order_book(save_file):
    stock_bid_ask_em_df = []
    while True:
        t = datetime.datetime.now().strftime("%H%M%S")
        if t < '092900':
            time.sleep(60)
            continue
        if t < '093000':
            time.sleep(1)
            continue
        if t >= '093000' and t <= '113000':
            if int(t[-2:]) % 3 == 0:
                _stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="000001")
                _stock_bid_ask_em_df.loc[:, 'time'] = t
                stock_bid_ask_em_df.append(_stock_bid_ask_em_df)
            time.sleep(1)
            continue
        
        if t > '113000' and t < '125900':
            time.sleep(60)
            continue
        if t > '125900' and t < '130000':
            time.sleep(1)
            continue
        if t >= '130000' and t <= '150000':
            if int(t[-2:]) % 3 == 0:
                _stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="000001")
                _stock_bid_ask_em_df.loc[:, 'time'] = t
                stock_bid_ask_em_df.append(_stock_bid_ask_em_df)
            time.sleep(1)
            continue
        
        if t > '150000':
            break
    
    stock_bid_ask_em_df = pd.concat(stock_bid_ask_em_df, axis=0)
    stock_bid_ask_em_df.to_csv(save_file, index=False)


In [ ]:
def test_real_price(save_file):
    stock_zh_a_spot_em_df = []
    save = False
    while True:
        t = datetime.datetime.now().strftime("%H%M%S")
        if t < '092900':
            time.sleep(60)
            continue
        if t < '093000':
            time.sleep(1)
            continue
        if t >= '093000' and t <= '113000':
            if int(t[-2:]) % 3 == 0:
                _stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
                _stock_zh_a_spot_em_df.loc[:, 'time'] = t
                stock_zh_a_spot_em_df.append(_stock_zh_a_spot_em_df)
            time.sleep(1)
            continue
        
        if t > '113000' and t < '125900':
            if not save:
                stock_zh_a_spot_em_df = pd.concat(stock_zh_a_spot_em_df, axis=0)
                stock_zh_a_spot_em_df.to_csv('save_file', index=False)
                stock_zh_a_spot_em_df = []
                save = True
            time.sleep(60)
            continue
        if t > '125900' and t < '130000':
            time.sleep(1)
            continue
        if t >= '130000' and t <= '150000':
            if int(t[-2:]) % 3 == 0:
                _stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
                _stock_zh_a_spot_em_df.loc[:, 'time'] = t
                stock_zh_a_spot_em_df.append(_stock_zh_a_spot_em_df)
            time.sleep(1)
            continue
        
        if t > '150000':
            if save:
                stock_zh_a_spot_em_df = pd.concat(stock_zh_a_spot_em_df, axis=0)
                stock_zh_a_spot_em_df.to_csv('save_file', index=False)
                stock_zh_a_spot_em_df = []
                save = False
            break
